In [ ]:
import docx
import os

data foramtting

In [ ]:
import docx
import os
import json

def extract_questions_answers(docx_path):
    print(f"Processing document: {docx_path}")
    
    doc = docx.Document(docx_path)
    
    conversations = []
    current_question = None
    current_answer = []
    bold_text_found = False
    
    for para in doc.paragraphs:
        para_text = para.text.strip()

        # Skip empty paragraphs or paragraphs that do not contain letters
        if not para_text or not any(char.isalpha() for char in para_text):
            continue

        # Check if the paragraph contains bold text (treated as a subtitle)
        is_bold = any(run.bold for run in para.runs)

        if is_bold:
            # If a previous question exists, append the conversation to the list
            if current_question and current_answer:
                conversations.append({
                    "messages": [
                        {"role": "system", "content": "You are a chatbot providing legal information."},
                        {"role": "user", "content": current_question},
                        {"role": "assistant", "content": ' '.join(current_answer).strip()}
                    ]
                })
                print(f"Conversation added: {current_question}")
            
            # Update the current question with the document name and bold text
            current_question = f"{os.path.basename(docx_path)} - {para_text}"
            print(f"New question identified: {current_question}")
            
            # Reset the current answer for the next block of text
            current_answer = []
            bold_text_found = True
        else:
            # If bold text has been found, start collecting paragraphs for the current answer
            if bold_text_found:
                current_answer.append(para_text)
                print(f"Answer in progress for: {current_question}")

    # Append the last conversation
    if current_question and current_answer:
        conversations.append({
            "messages": [
                {"role": "system", "content": "You are a chatbot providing legal information."},
                {"role": "user", "content": current_question},
                {"role": "assistant", "content": ' '.join(current_answer).strip()}
            ]
        })
        print(f"Final conversation added: {current_question}")

    print(f"Finished processing document: {docx_path}")
    return conversations

def process_docx_files(docx_folder, output_folder):
    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        print(f"Created output folder: {output_folder}")

    # Iterate over all .docx files in the folder
    for filename in os.listdir(docx_folder):
        if filename.endswith('.docx'):
            docx_path = os.path.join(docx_folder, filename)
            print(f"\nStarting processing of file: {filename}")
            
            conversations = extract_questions_answers(docx_path)

            # Save the extracted data into a JSON Lines file with the same base name
            output_filename = f"{os.path.splitext(filename)[0]}.jsonl"
            output_path = os.path.join(output_folder, output_filename)

            with open(output_path, 'w', encoding='utf-8') as f:
                for conversation in conversations:
                    f.write(f"{json.dumps(conversation, ensure_ascii=False)}\n")
            
            print(f"Saved JSON Lines to {output_path}")
    
    print(f"\nAll documents processed. JSON Lines files saved to {output_folder}")

# Paths to the folder containing .docx law documents and the output folder for JSON files
docx_folder = r'../documents_docx'
output_folder = r'../documents_json'

# Process all .docx files and save the outputs as JSON Lines
process_docx_files(docx_folder, output_folder)

print(f"\nAll tasks completed.")
